In [52]:
from torchvision import datasets, models, transforms
import dill as pickle
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import torch
from models import DenseNet121
import joblib
import torch.nn as nn
from utililties import *
from torch.utils.data import Dataset, DataLoader, Subset

nrows = 256
ncolumns = 256

dbfile = open('sample.pickle', 'rb')      
db = pickle.load(dbfile) 

image_path = f"data/training/COVID/2020.01.24.919183-p27-132.png"

img = cv2.imread(image_path)
t = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.cpu().numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

# matplotlib_imshow(t)

data = torch.from_numpy(t).float()
# print(data)
print (data.shape)
X = []
if t.ndim>=3:
    X.append(np.moveaxis(cv2.resize(t[:,:,:3], (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))
else:
    smimg= cv2.cvtColor(t,cv2.COLOR_GRAY2RGB)
    X.append(np.moveaxis(cv2.resize(smimg, (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))

x = np.array(X)
print(torch.from_numpy(x).shape)

model_main = DenseNet121(num_classes=2,pretrained=True)
checkpoint0 = torch.load("Model_densenet121_state.pth")
model_main.load_state_dict(checkpoint0)

clf = joblib.load('classifier_model.sav')
model_main.eval()
model_main.fc = nn.Identity()

image_transforms = transforms.Compose([
        transforms.Lambda(lambda x: x/255),
        transforms.ToPILImage(), 
       #transforms.Resize((224, 224)),
        transforms.Resize((230, 230)),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.45271412, 0.45271412, 0.45271412],
                             [0.33165374, 0.33165374, 0.33165374])
     ])
    
dataset = MyDataset_test(x,image_transforms)

for param in model_main.parameters():
             param.requires_grad_(False)

dataloader = DataLoader(
dataset,
batch_size=16,
pin_memory=True,worker_init_fn=np.random.seed(7), drop_last=False)

y_pred2=[]
for inputs in dataloader:
    outputs = model_main(inputs)
    preds = clf.predict(outputs)
    
    for ii in range(len(preds)):
        if preds[ii] > 0.5:
            y_pred2.append('COVID')
        #print('covid')
        else:
            y_pred2.append('NonCOVID')
        #print('noncovid')

print(y_pred2)






torch.Size([551, 725, 3])
torch.Size([1, 3, 256, 256])
['COVID']


/opt/miniconda3/envs/treehax2/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/miniconda3/envs/treehax2/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
